<a href="https://colab.research.google.com/github/drizedunn/Food-Preference/blob/main/food_preference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Preference
In this project, we predict the food type of individuals. It can either be western food or tradition food.

In [ ]:
# Importing the needed libraries

# !pip install shap
# !pip install colorama

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from colorama import Style, Fore, Back
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from joblib import dump

from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score

import shap
import warnings

plt.style.use('seaborn')
warnings.filterwarnings('ignore')
%matplotlib inline

### Data Collection and loading

The data is a food survey conducted among participants from different countries and demography which can be found [here](https://www.kaggle.com/datasets/vijayashreer/food-preferences)

In [2]:
# reading the data
# data = pd.read_csv('/content/drive/MyDrive/Datasets/Food_Preference.csv')
data = pd.read_csv('./dataset/Food_Preference.csv')

In [3]:
#Displaying the first five rows of the data
data.head()

,Timestamp,Participant_ID,Gender,Nationality,Age,Food,Juice,Dessert
0,2019/05/07 2:59:13 PM GMT+8,FPS001,Male,Indian,24,Traditional food,Fresh Juice,Maybe
1,2019/05/07 2:59:45 PM GMT+8,FPS002,Female,Indian,22,Western Food,Carbonated drinks,Yes
2,2019/05/07 3:00:05 PM GMT+8,FPS003,Male,Indian,31,Western Food,Fresh Juice,Maybe
3,2019/05/07 3:00:11 PM GMT+8,FPS004,Female,Indian,25,Traditional food,Fresh Juice,Maybe
4,2019/05/07 3:02:50 PM GMT+8,FPS005,Male,Indian,27,Traditional food,Fresh Juice,Maybe
